# 4 — PCA for Music Features (Theory + Interpretation)

**Goal:** Understand and *use* PCA to visualize Spotify audio features, reduce noise/correlation, and interpret the main axes of variation.

**You’ll learn:**
- Intuition: what PCA optimizes (variance) and why it helps clustered data
- How to read **explained variance** (scree & cumulative plots)
- How to **interpret loadings** (which features make up PC1/PC2)
- Visualize K-Means clusters in 2D PCA space and connect to *moods/playlists*

> We keep PCA for **visualization & interpretation**. We do **not** force clustering in PCA space here.

## 0) Imports & Setup

In [ ]:

import numpy as np
import pandas as pd
import re
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

plt.rcParams['figure.figsize'] = (7,5)
RNG = np.random.RandomState(42)


## 1) Load data (and optional precomputed clusters)
Expected CSV: `../data/spotify_5000_songs.csv`

**Tip:** If you already ran the K-Means notebook, we’ll try to load `../reports/kmeans20_quantile_assignments.csv` so we can color points by cluster. Otherwise, we’ll compute a quick KMeans@20 here.

In [ ]:

DATA = Path("../data/spotify_5000_songs.csv")
assert DATA.exists(), f"Missing data file at {DATA}. Place your CSV there."

def clean_col(c):
    s = re.sub(r"\s+", " ", str(c)).strip()
    return s.split(" ")[0]

df_raw = pd.read_csv(DATA)
df = df_raw.copy()
df.columns = [clean_col(c) for c in df.columns]

FEATURES = ['danceability','energy','acousticness','instrumentalness','liveness','valence',
            'tempo','speechiness','loudness','duration_ms','key','mode','time_signature']
available = [c for c in FEATURES if c in df.columns]
X = df[available].apply(pd.to_numeric, errors='coerce').dropna()

print("Using features:", available, "| Shape:", X.shape)

assign_path = Path("../reports/kmeans20_quantile_assignments.csv")
labels = None
if assign_path.exists():
    df_assign = pd.read_csv(assign_path)
    if 'cluster' in df_assign.columns and len(df_assign)==len(df):
        labels = df_assign['cluster'].values[:len(X)]
        print("Loaded clusters from:", assign_path)


## 2) Why PCA? (Intuition)
- Many audio features are **correlated** (e.g., energy & loudness). PCA finds new axes (PCs) that are **uncorrelated** and capture the most variance.
- PC1 captures the largest direction of variation; PC2 the next largest **orthogonal** direction, etc.
- This is useful for: plotting in 2D, de-noising, and understanding the *main musical dimensions* (e.g., **energy** vs **valence**).

## 3) Scale features before PCA
To avoid features with large ranges dominating, we’ll scale. We’ll use **QuantileTransformer** (good for skew). You can toggle to **StandardScaler** to compare.

In [ ]:

use_quantile = True

if use_quantile:
    scaler = QuantileTransformer(output_distribution='normal', n_quantiles=min(1000, len(X)), random_state=42)
else:
    scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled[:3]


## 4) Fit PCA and inspect explained variance
**Explained variance ratio** tells us how much of the dataset's variability each PC captures. Use the **scree plot** to spot elbows and the **cumulative** curve to see how many PCs explain, say, 80–95% variance.

In [ ]:

# Fit more PCs than 2 so we can inspect variance structure
n_components = min(12, X_scaled.shape[1])
pca = PCA(n_components=n_components, random_state=42).fit(X_scaled)

explained = pca.explained_variance_ratio_
cum = explained.cumsum()

fig, ax = plt.subplots()
ax.plot(range(1, n_components+1), explained, marker='o')
ax.set_xlabel('PC'); ax.set_ylabel('Explained variance ratio')
ax.set_title('Scree plot')
plt.show()

fig, ax = plt.subplots()
ax.plot(range(1, n_components+1), cum, marker='o')
ax.axhline(0.8, linestyle='--'); ax.axhline(0.9, linestyle='--')
ax.set_xlabel('PC'); ax.set_ylabel('Cumulative explained variance')
ax.set_title('Cumulative variance')
plt.show()

pd.DataFrame({
    'PC': [f'PC{i}' for i in range(1, n_components+1)],
    'explained_ratio': explained,
    'cumulative_ratio': cum
})


## 5) Interpret PC loadings (feature contributions)
**Loadings** indicate how each original feature contributes to a PC. Large positive/negative values mean strong influence.
Interpreting PC1/PC2 helps translate axes into musical language (e.g., PC1 ~ energy/loudness → ‘intensity axis’).

In [ ]:

loadings = pd.DataFrame(pca.components_.T,
                        index=X.columns[:pca.components_.shape[1]],
                        columns=[f'PC{i}' for i in range(1, n_components+1)])
# Show top magnitudes for PC1 and PC2
def top_loadings(pc, n=8):
    s = loadings[pc].abs().sort_values(ascending=False).head(n).index
    return loadings.loc[s, pc].sort_values(ascending=False)

top_pc1 = top_loadings('PC1', 8)
top_pc2 = top_loadings('PC2', 8)

print("Top PC1 loadings:")
display(top_pc1)
print("\nTop PC2 loadings:")
display(top_pc2)


### Optional: bar plots of PC1/PC2 loadings

In [ ]:

for pc in ['PC1','PC2']:
    s = loadings[pc].sort_values()
    plt.figure(figsize=(8,4))
    s.plot(kind='barh')
    plt.title(f'{pc} loadings (all features)')
    plt.tight_layout()
    plt.show()


## 6) 2D PCA projection colored by cluster
If we have labels (from K-Means k=20), we’ll color by cluster to *see* separation. If not, we’ll quickly compute K-Means here.

In [ ]:

X2 = PCA(n_components=2, random_state=42).fit_transform(X_scaled)

if labels is None:
    km = KMeans(n_clusters=20, n_init=10, random_state=42).fit(X_scaled)
    labels = km.labels_
    print("No precomputed labels found; computed KMeans(k=20) on the fly.")

plt.scatter(X2[:,0], X2[:,1], c=labels, s=5)
plt.title('PCA 2D — colored by K-Means cluster')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()


**How to read this map**
- **Islands/blobs** of one color suggest clear playlist ‘moods’.
- **Bridges** or mixed regions might be transitional tracks.
- If everything overlaps, consider different scaling, features, or algorithms (e.g., Agglomerative).

## 7) Save PCA outputs for reuse
These can be included in your README or slides and help with reproducibility.

In [ ]:

OUT = Path("../reports")
OUT.mkdir(parents=True, exist_ok=True)

# Save 2D projection and labels (if present)
pca_df = pd.DataFrame({'PC1': X2[:,0], 'PC2': X2[:,1], 'cluster': labels})
pca_path = OUT / "pca2_projection_kmeans20.csv"
pca_df.to_csv(pca_path, index=False)

print(f"Saved: {pca_path}")


---
## 8) Takeaways
- PCA reveals **major axes of musical variation** (often energy/loudness vs. valence/tempo).
- Use **explained variance** to decide how many PCs you need for visualization/reporting (not for clustering here).
- Loadings **translate math → music**: they tell you what each axis means.
- The 2D PCA map is a great storytelling tool for Moosic editors and stakeholders.